In [1]:
import numpy as np

In [2]:
shape = (100, 100, 100)
a = np.random.random(shape)
b = np.random.random(shape)
c = np.random.random(shape)

In [3]:
def f(a, b, c):
    return a + b + c

In [4]:
%%timeit

res = f(a, b, c)

2.66 ms ± 22.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
def pythonf(a, b, c):
    result = np.zeros(a.shape)
    for x in range(a.shape[0]):
        for y in range(a.shape[1]):
            for t in range(a.shape[2]):
                result[x, y, t] = a[x, y, t] + b[x, y, t] + c[x, y, t]
    return result

print((pythonf(a, b, c) == f(a, b, c)).all())

True


In [6]:
%%timeit

res = pythonf(a, b, c)

636 ms ± 4.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
vectorizedf = np.vectorize(f)
print((vectorizedf(a, b, c) == f(a, b, c)).all())

True


In [8]:
%%timeit

res = vectorizedf(a, b, c)

237 ms ± 1.29 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
from numba import jit

@jit
def numbaf(a, b, c):
    result = np.zeros(a.shape)
    for x in range(a.shape[0]):
        for y in range(a.shape[1]):
            for t in range(a.shape[2]):
                result[x, y, t] = a[x, y, t] + b[x, y, t] + c[x, y, t]
    return result

print((numbaf(a, b, c) == f(a, b, c)).all())

True


In [10]:
%%timeit

res = numbaf(a, b, c)

2.69 ms ± 14.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
%load_ext cython

In [12]:
%%cython -a

import numpy as np
cimport numpy as np
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef np.float64_t[:, :, :] cythonf(np.float64_t[:, :, :] a, 
                                    np.float64_t[:, :, :] b, 
                                    np.float64_t[:, :, :] c):
    cdef int w = a.shape[0]
    cdef int h = a.shape[1]
    cdef int d = a.shape[2]
    cdef np.float64_t[:, :, :] result = np.zeros((w, h, d), dtype = np.float64)
    cdef int x, y, z
    for x in range(w):
        for y in range(h):
            for t in range(d):
                result[x, y, t] = a[x, y, t] + b[x, y, t] + c[x, y, t]
    return result

In [13]:
print((cythonf(a, b, c) == f(a, b, c)).all())

True


In [14]:
%%timeit

res = cythonf(a, b, c)

2.68 ms ± 16.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
